**Source:** Forré & Mooij (2019) - "Causal Calculus in the Presence of Cycles, Latent Confounders and Selection Bias"

---

## Algorithm Pseudocode

### Function: ID (Main Function)

$$
\begin{align*}
&\textbf{1: function } \mathbf{ID}(G, \mathbf{Y}, \mathbf{W}, P(\mathbf{V} | do(\mathbf{J}))) \\
&\textbf{2: require: } \mathbf{Y} \subseteq \mathbf{V}, \mathbf{W} \subseteq \mathbf{V}, \mathbf{Y} \cap \mathbf{W} = \emptyset \\
&\textbf{3: } H \leftarrow \text{Anc}^{G_{\mathbf{V} \setminus \mathbf{W}}}(\mathbf{Y}) \\
&\textbf{4: } \textbf{for } C \in \text{CD}(H) \textbf{ do} \\
&\textbf{5: } \quad Q[C] \leftarrow \text{IDCD}(G, C, \text{Cd}^G(C), Q[\text{Cd}^G(C)]) \\
&\textbf{6: } \quad \textbf{if } Q[C] = \text{FAIL} \textbf{ then} \\
&\textbf{7: } \quad\quad \textbf{return } \text{FAIL} \\
&\textbf{8: } \quad \textbf{end if} \\
&\textbf{9: } \textbf{end for} \\
&\textbf{10: } Q[H] \leftarrow \left[ \bigotimes_{C \in \text{CD}(H)} \right] Q[C] \\
&\textbf{11: } \textbf{return } P(\mathbf{Y} | do(\mathbf{J}, \mathbf{W})) = \int Q[H] \, d\mathbf{x}_{H \setminus \mathbf{Y}} \\
&\textbf{12: } \textbf{end function}
\end{align*}
$$

---

### Function: IDCD (Recursive Helper Function)

$$
\begin{align*}
&\textbf{13: function } \mathbf{IDCD}(G, \mathbf{C}, \mathbf{D}, Q[\mathbf{D}]) \\
&\textbf{14: require: } \mathbf{C} \subseteq \mathbf{D} \subseteq \mathbf{V}, \text{CD}(G_{\mathbf{D}}) = \{\mathbf{D}\} \\
&\textbf{15: } A \leftarrow \text{Anc}^{G[\mathbf{D}]}(\mathbf{C}) \cap \mathbf{D} \\
&\textbf{16: } Q[A] \leftarrow \int Q[\mathbf{D}] \, d(\mathbf{x}_{\mathbf{D} \setminus A}) \\
&\textbf{17: } \textbf{if } A = \mathbf{C} \textbf{ then} \\
&\textbf{18: } \quad \textbf{return } Q[A] \\
&\textbf{19: } \textbf{else if } A = \mathbf{D} \textbf{ then} \\
&\textbf{20: } \quad \textbf{return } \text{FAIL} \\
&\textbf{21: } \textbf{else if } \mathbf{C} \subset A \subset \mathbf{D} \textbf{ then} \\
&\textbf{22: } \quad \textbf{for } S \in \mathcal{S}(G[A]) \text{ s.t. } S \subseteq \text{Cd}^{G[A]}(\mathbf{C}) \textbf{ do} \\
&\textbf{23: } \quad\quad R_A[S] \leftarrow P(S | \text{Pred}^G_<(S) \cap A, do(\mathbf{J} \cup \mathbf{V} \setminus A)) \\
&\textbf{24: } \quad \textbf{end for} \\
&\textbf{25: } \quad Q[\text{Cd}^{G[A]}(\mathbf{C})] \leftarrow \bigotimes_{\substack{S \in \mathcal{S}(G[A]) \\ S \subseteq \text{Cd}^{G[A]}(\mathbf{C})}} R_A[S] \\
&\textbf{26: } \quad \textbf{return } \text{IDCD}(G, \mathbf{C}, \text{Cd}^{G[A]}(\mathbf{C}), Q[\text{Cd}^{G[A]}(\mathbf{C})]) \\
&\textbf{27: } \textbf{end if} \\
&\textbf{28: } \textbf{end function}
\end{align*}
$$

### Line 1: Function Declaration - ID Algorithm

$$
\textbf{1: function } \mathbf{ID}(G, \mathbf{Y}, \mathbf{W}, P(\mathbf{V} | do(\mathbf{J})))
$$

---

#### Mathematical Definition

**Function Signature:**
```
ID(G, Y, W, P(V | do(J))) → P(Y | do(J, W)) or FAIL
```

| Parameter | Type | Meaning | Constraints |
|-----------|------|---------|-------------|
| $G$ | Directed Mixed Graph | Causal graph (may have cycles & latent confounders) | - |
| $\mathbf{Y}$ | Set of Variables | Target variables (outcomes to identify) | $\mathbf{Y} \subseteq \mathbf{V}$ |
| $\mathbf{W}$ | Set of Variables | Intervention variables (treatments) | $\mathbf{W} \subseteq \mathbf{V}$ |
| $P(\mathbf{V} \mid do(\mathbf{J}))$ | Probability Distribution | Observational data under background interventions | Given/observed |

**Returns:**
- Expression for $P(\mathbf{Y} \mid do(\mathbf{J}, \mathbf{W}))$ computable from $P(\mathbf{V} \mid do(\mathbf{J}))$
- OR "FAIL" if causal effect is not identifiable

**The Question Being Answered:**
> "Can I predict $\mathbf{Y}$ under intervention $do(\mathbf{W})$ using only observational data, even when the causal graph has **feedback loops**?"

## Statistical & Causal Assumptions

### 1. **Causal Markov Condition**
- **What**: Each variable is independent of non-descendants given its parents
- **Formula**: $P(\mathbf{V}) = \prod_{X \in \mathbf{V}} P(X \mid \text{Pa}^G(X))$
- **Why**: Allows factorization of joint distribution by graph structure
- **Enables**: Decomposing complex distributions into simpler conditional probabilities

### 2. **Faithfulness**
- **What**: All independencies in data arise from graph structure only (no "accidental" cancellations)
- **Why**: Ensures $G$ accurately represents the causal mechanism, not just a compatible model
- **Fails when**: Two causal paths exactly cancel (rare in practice)

### 3. **Latent Confounders via Bidirected Edges**
- **Notation**: $X \leftrightarrow Y$ represents hidden $U$ with $U \to X$ and $U \to Y$
- **Why**: Handles unmeasured confounding without assuming we observed everything
- **Key difference**: RELAXES standard causal sufficiency assumption from acyclic ID

### 4. **Stable Equilibria in Cycles (Unique to Cyclic Graphs)**
- **What**: Feedback systems reach unique stable equilibrium
- **Mathematical**: System of structural equations has unique fixed-point solution
- **Example**: In gene regulation with negative feedback ($X \dashv Y \to X$), protein levels stabilize
- **Why critical**: Without stability, $P(\mathbf{V})$ undefined in cyclic systems
- **Not needed**: In standard (acyclic) causal graphs

### 5. **Structural Causal Model (SCM)**
- **What**: Each variable has structural equation $X := f_X(\text{Pa}^G(X), U_X)$
- **Intervention semantics**: $do(X=x)$ replaces $f_X$ with constant function: $X := x$
- **Why**: Defines what "intervention" means mathematically (Pearl's do-operator)

### 6. **Correct Background Interventions**
- **What**: Background conditions $\mathbf{J}$ are accurately controlled/known
- **Why**: Algorithm builds new interventional queries from existing data $P(\mathbf{V} \mid do(\mathbf{J}))$
- **Example**: If $\mathbf{J}$ = "glucose = 10mM, 37°C", these must be experimentally fixed

---

## Plain English Explanation

### What This Line Does
Declares the main identification algorithm for causal graphs **with feedback loops** (cycles). This extends the classical ID algorithm (acyclic only) to handle circular causality.

### The Four Inputs

**1. Graph $G$ - Your Causal Model:**
- Shows which variables affect others (directed edges: $X \to Y$)
- **Can have cycles**: $X \to Y \to Z \to X$ (feedback regulation)
- **Can have latent confounders**: $X \leftrightarrow Y$ (unmeasured common cause)
- Example: Gene regulatory network with feedback inhibition

**2. Outcomes $\mathbf{Y}$ - What You Want to Predict:**
- Must be measured/observable variables
- Can be single or multiple variables
- Example: {lysine concentration, cell growth rate}

**3. Treatments $\mathbf{W}$ - What You're Manipulating:**
- Must be variables you can control/intervene on
- Cannot overlap with $\mathbf{Y}$ (can't ask "effect of X on X")
- Example: {gene knockout, temperature change}

**4. Data $P(\mathbf{V} \mid do(\mathbf{J}))$ - Your Observations:**
- Joint distribution of all variables
- Already collected under specific conditions $\mathbf{J}$
- Example: Wild-type transcriptomics/metabolomics in minimal media

### What It Returns

**If identifiable:**
- Formula showing how to compute $P(\mathbf{Y} \mid do(\mathbf{W}))$ from your observational data
- You can answer "what if" questions without doing new experiments

**If not identifiable:**
- Returns "FAIL" 
- Need additional experiments or different interventions

## Key Takeaways/Summary

1. **Main entry point** for cyclic identification algorithm
2. **Extends classical ID** from acyclic to cyclic graphs
3. **Requires stability assumption** for feedback loops (unique to cyclic case)
4. **Takes observational data**, returns interventional distribution (or FAIL)
5. **Enables causal inference** in systems with feedback regulation